# ESM2-LoRA 定向进化工作流：基于 Gibbs 采样的序列漂移

本 Notebook 实现了**“生成式”**的蛋白改造策略（Strategy 1: In silico Sequence Drift）。

**核心原理**：
利用微调后的 LoRA 模型（已习得嗜热/耐碱等特性），对野生型蛋白进行**迭代式掩码重构 (Iterative Masking & Refilling)**。
这类似于在计算机中模拟自然界的“随机突变-选择”过程，但这里的“选择”是由 ESM2 模型根据其学到的进化概率完成的。

**适用场景**：
1. 寻找深度的组合突变（不仅是单点）。
2. 探索序列的“隐空间”，让蛋白在保持核心结构的同时“漂移”向更稳定的状态。
3. 为构建突变库提供高潜力的候选序列。

In [1]:
import os

# 设置 HuggingFace 镜像
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

import torch
import random
from tqdm import tqdm
import copy
import pandas as pd
from transformers import AutoTokenizer, EsmForMaskedLM
from peft import PeftModel



### 1. 全局参数设置 (User Configuration)
**请在此处设置您的模型路径和目标蛋白序列。**

In [2]:
# ================= 模型路径 =================
# 基础模型 (通常不用改)
BASE_MODEL_NAME = "facebook/esm2_t33_650M_UR50D"

# 您的 LoRA 模型路径 (训练输出的 final_model 文件夹路径)
# 示例: "./esm2_lora_output/final_model"
LORA_MODEL_PATH = "/esm2_lora_ph5.0_less/final_model"

# ================= 目标蛋白 =================
# 待改造的野生型序列 (Wild Type)
TARGET_SEQUENCE = "SLKD"

# ================= 进化参数 (关键) =================
# 1. 进化轮数 (Steps): 建议 50-200。轮数越多，序列改变越大。
NUM_STEPS = 50

# 2. 掩码比例 (Mask Ratio): 每次迭代随机遮挡多少比例的氨基酸。建议 0.1 - 0.15。
# 0.1 意味着每次改动较温和，0.2 意味着剧烈变动。
MASK_RATIO = 0.2

# 3. 采样多样性 (Top-K): 每次填空时，从概率最高的 K 个氨基酸里抽签。
# K=1: 贪婪模式 (总是选概率最大的，结果最稳定但最保守)。
# K=5: 推荐值 (允许一定随机性，能发现新颖突变)。
TOP_K = 5

# 4. 并行链数 (Parallel Chains): 同时模拟几条独立的进化路径？
# 建议 3-5 条，方便对比不同路径产生的突变。
NUM_CHAINS = 500
BATCH_SIZE = 64 # 每次处理的链数，建议根据显存调整

# 5. 固定位点 (Optional): 如果某些位点绝对不能动（如催化中心），请填入列表 (1-based)。
# 示例: FROZEN_POSITIONS = [10, 11, 12]  (表示第10,11,12位保持不变)
FROZEN_POSITIONS = []

### 2. 模型加载
自动加载基础模型并挂载您的 LoRA 适配器。

In [3]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()
print(f"🚀 正在使用计算设备: {device}")

# 加载 Tokenizer
print("⏳ 正在加载 Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# 加载基础模型
print("⏳ 正在加载基础 ESM2 模型 (这可能需要一点时间)...")
base_model = EsmForMaskedLM.from_pretrained(BASE_MODEL_NAME)

# 挂载 LoRA
print(f"🔄 正在挂载 LoRA 适配器: {LORA_MODEL_PATH} ...")
model = PeftModel.from_pretrained(base_model, LORA_MODEL_PATH)
model.to(device)
model.eval() # 开启推理模式

# 🚀 只有在 NVIDIA GPU (Linux) 上才开启这个
if torch.cuda.is_available():
    print("🔥 检测到 NVIDIA GPU，正在启用 torch.compile 编译加速...")
    try:
        # 这会让第一次运行稍微慢一点（编译），但之后的 500 步循环会飞快
        model = torch.compile(model) 
    except Exception as e:
        print(f"⚠️ 编译失败，回退到普通模式: {e}")

print("✅ 模型加载完毕，进化引擎已就绪。")

🚀 正在使用计算设备: mps
⏳ 正在加载 Tokenizer...
⏳ 正在加载基础 ESM2 模型 (这可能需要一点时间)...
🔄 正在挂载 LoRA 适配器: /Users/zhanghaonan/Documents/ML/protein_galaxy/esm2_lora/esm2_lora_ph5.0_less/final_model ...
✅ 模型加载完毕，进化引擎已就绪。


### 3. 定义 Gibbs 采样进化函数
这是核心算法部分。它执行“遮挡 -> 预测 -> 采样 -> 更新”的循环。

In [ ]:
def evolve_batch_tensor_optimized(start_seq, num_chains, num_steps, patience=10):
    """
    极致优化版：包含 Early Stopping 和 inference_mode
    patience: 如果序列连续多少步没有采纳新突变(或变化极小)，则认为收敛
    """
    # 强制使用 inference_mode (比 no_grad 更快一点点)
    # 强制 FP16
    initial_inputs = tokenizer(start_seq, return_tensors="pt").to(device)
    input_ids_base = initial_inputs["input_ids"]
    seq_len = len(start_seq)
    mask_token_id = tokenizer.mask_token_id
    
    final_sequences = []
    
    # 动态调整 Batch Size (M4 32G 建议 64-100，如果 200 慢可能是内存交换导致，建议设为 64)
    OPTIMAL_BATCH_SIZE = BATCH_SIZE
    num_batches = (num_chains + OPTIMAL_BATCH_SIZE - 1) // OPTIMAL_BATCH_SIZE
    
    print(f"🚀 极速进化模式: Steps={num_steps} | Batch={OPTIMAL_BATCH_SIZE}")
    
    for batch_idx in range(num_batches):
        current_batch_size = min(OPTIMAL_BATCH_SIZE, num_chains - batch_idx * OPTIMAL_BATCH_SIZE)
        
        # [Batch, Length]
        current_input_ids = input_ids_base.repeat(current_batch_size, 1)
        
        # 用于 Early Stopping 的记录
        # 记录上一步的序列，用于对比
        prev_input_ids = current_input_ids.clone()
        no_change_counter = torch.zeros(current_batch_size, device=device)
        
        # 开启 inference_mode 
        with torch.inference_mode(): 
            for step in tqdm(range(num_steps), desc=f"Batch {batch_idx+1}/{num_batches}", leave=False):
                
                # --- 1. 生成 Mask (增加了保护机制，避免 Mask 太多导致崩坏) ---
                # 动态 Mask 比例：随着步数增加，Mask 比例逐渐降低（模拟退火）
                # 前 50% 步数用高 Mask (0.2)，后 50% 用低 Mask (0.1) 精修
                current_mask_ratio = MASK_RATIO if step < num_steps * 0.5 else MASK_RATIO * 0.5
                
                rand_matrix = torch.rand(current_input_ids.shape, device=device)
                mask_bool = rand_matrix < current_mask_ratio
                mask_bool[:, 0] = False; mask_bool[:, -1] = False # 保护 CLS/EOS
                
                # 保护固定位点
                if FROZEN_POSITIONS:
                    for pos in FROZEN_POSITIONS:
                        mask_bool[:, pos] = False
                
                masked_input_ids = current_input_ids.clone()
                masked_input_ids[mask_bool] = mask_token_id
                
                # --- 2. 推理 ---
                logits = model(input_ids=masked_input_ids).logits
                
                # --- 3. 采样 ---
                probs = torch.softmax(logits, dim=-1)
                top_k_probs, top_k_ids = torch.topk(probs, TOP_K)
                top_k_probs = top_k_probs / torch.sum(top_k_probs, dim=-1, keepdim=True)
                
                # 展平采样
                flat_probs = top_k_probs.view(-1, TOP_K)
                flat_ids = top_k_ids.view(-1, TOP_K)
                sampled_indices = torch.multinomial(flat_probs, 1)
                sampled_token_ids = torch.gather(flat_ids, 1, sampled_indices).view(current_batch_size, -1)
                
                # --- 4. 更新 ---
                current_input_ids = torch.where(mask_bool, sampled_token_ids, current_input_ids)
                
                # --- 5. Early Stopping 检查 (可选优化) ---
                # 检查哪些序列这一步完全没变 (或者变回去了)
                # 注意：这里为了速度，每 10 步检查一次
                if step % 10 == 0:
                    is_same = torch.all(current_input_ids == prev_input_ids, dim=1)
                    no_change_counter[is_same] += 1
                    no_change_counter[~is_same] = 0 # 重置
                    prev_input_ids = current_input_ids.clone()
                    
                    # 如果所有序列都连续 5 次检测(50步)没变化，提前结束当前 Batch
                    if torch.all(no_change_counter >= 5):
                        tqdm.write(f"  ⚡️ Batch {batch_idx+1} 在 Step {step} 提前收敛，跳过后续步骤。")
                        break

        # 解码
        for i in range(current_batch_size):
            seq_str = tokenizer.decode(current_input_ids[i], skip_special_tokens=True).replace(" ", "")
            final_sequences.append(seq_str)
            
    return final_sequences

### 4. 运行进化
根据设置的 `NUM_CHAINS` 启动多条并行进化链，生成不同的候选序列。

In [6]:
# 确保使用半精度 (Float16) 以节省显存并加速 (M4 MPS 支持良好)
# 注意：如果之前加载模型没用半精度，这里转换一下
model.half() 
print("⚡️ 模型已转换为 FP16 半精度模式")

# 运行并行进化
# 这里直接生成所有链
evolved_sequences = evolve_batch_tensor_optimized(
    start_seq=TARGET_SEQUENCE, 
    num_chains=NUM_CHAINS,  # 1000
    num_steps=NUM_STEPS     # 500
)

print(f"✅ 完成！共生成 {len(evolved_sequences)} 条序列。")

⚡️ 模型已转换为 FP16 半精度模式
🚀 极速进化模式: Steps=50 | Batch=64


✅ 完成！共生成 500 条序列。


### 5. 结果分析与可视化
对比野生型 (WT) 和生成的变体，提取具体的突变位点 (例如 `A26R`)。

In [7]:
def compare_sequences(wt, mut_seq):
    """比较两条序列，返回突变列表"""
    mutations = []
    for i, (aa1, aa2) in enumerate(zip(wt, mut_seq)):
        if aa1 != aa2:
            # 格式: 野生型 + 位置(1-based) + 突变型
            mutations.append(f"{aa1}{i+1}{aa2}")
    return mutations

# 汇总结果
results_data = []

for i, seq in enumerate(evolved_sequences):
    muts = compare_sequences(TARGET_SEQUENCE, seq)
    results_data.append({
        "Chain_ID": i + 1,
        "Mutation_Count": len(muts),
        "Mutations": ", ".join(muts),
        "Full_Sequence": seq
    })

# 转换为 DataFrame 展示
df_results = pd.DataFrame(results_data)

print("\n📊 进化结果汇总:")
display(df_results[[ "Chain_ID", "Mutation_Count", "Mutations"]])

# 统计最热门的突变
all_muts = []
for m_str in df_results["Mutations"]:
    if m_str:
        all_muts.extend(m_str.split(", "))

if all_muts:
    from collections import Counter
    counts = Counter(all_muts)
    print("\n🔥 [Hotspots] 出现频率最高的突变 (共识突变):")
    for mut, count in counts.most_common(10):
        print(f"   {mut}: 出现 {count}/{NUM_CHAINS} 次")
else:
    print("\n没有发生任何突变，请检查 MASK_RATIO 是否太低或 TOP_K 是否为 1。")


📊 进化结果汇总:


,Chain_ID,Mutation_Count,Mutations
0,1,51,"V1M, D2E, N3D, K4E, F5I, N6K, K7E, E8L, Q9I, Q..."
1,2,51,"V1M, D2S, N3E, F5I, N6D, K7P, Q9I, Q10R, N11E,..."
2,3,52,"V1M, D2E, N3E, K4E, F5L, N6E, K7E, Q9L, Q10E, ..."
3,4,51,"V1M, D2T, N3K, F5Q, N6K, K7E, Q9M, Q10K, N11K,..."
4,5,49,"V1M, N3K, K4E, F5N, N6L, K7I, E8N, Q9K, Q10L, ..."
...,...,...,...
495,496,52,"V1M, D2E, N3L, F5L, N6K, K7E, E8K, Q9L, Q10L, ..."
496,497,50,"V1M, D2A, N3D, K4D, F5A, N6D, K7A, E8A, Q9S, Q..."
497,498,55,"V1M, D2T, N3D, K4T, F5P, N6P, K7D, E8D, Q9P, Q..."
498,499,52,"V1M, D2K, N3K, F5I, N6K, E8I, Q9K, Q10I, N11E,..."



🔥 [Hotspots] 出现频率最高的突变 (共识突变):
   V1M: 出现 425/500 次
   P57K: 出现 180/500 次
   N3K: 出现 165/500 次
   A42K: 出现 158/500 次
   A29K: 出现 152/500 次
   A46E: 出现 152/500 次
   A56K: 出现 152/500 次
   E8K: 出现 151/500 次
   N28E: 出现 148/500 次
   D53K: 出现 148/500 次


### 6. 保存结果
将生成的序列保存到 Excel 文件中，以便后续进行结构验证 (AlphaFold) 或实验筛选。

In [ ]:
output_file = "evolved_variants.xlsx"
df_results.to_excel(output_file, index=False)
print(f"💾 结果已保存至: {output_file}")
print("建议下一步：")
print("1. 将生成的序列输入 AlphaFold/ESMFold 预测结构。")
print("2. 剔除 pLDDT 分数显著降低的序列。")
print("3. 重点关注 '共识突变' (Hotspots)，这些通常是 LoRA 模型最确定的优化点。")